# 表达式模板教程

本页解释mshadow如何工作。 mshadow后面的主要技巧是表达式模板。我们将解释它将如何影响编译代码的性能。表达式模板是C ++矩阵库的主要技巧，例如Eigen，GSL，boost.uBLAS。

## 如何编写高效的机器学习代码
在我们开始之前，让我们想一想上面的问题。假设我们要实现如下更新规则  
```c
weight =  - eta * (grad + lambda * weight);
```
其中weight和grad是长度为n的向量。当你选择C ++作为你的编程语言，我想主要关注的是效率。在大多数C / C ++程序中有一个重要的原则：
* 预分配必要的内存，在运行期间没有临时内存分配。  
例如以下的示例代码
```c
void UpdateWeight (const float *grad, float eta, float lambda, int n, float *weight) {
      for (int i = 0; i < n; ++i) {
        weight[i] =  - eta * (grad[i] + lambda * weight[i]);
      }
}
```
该函数接受预分配的空间grad和weight，并运行计算。写这些函数很简单，但是，当我们重复写它们时，它可能很烦人。所以问题是，我们可以写出如下代码，并获得与以前的代码相同的性能？
```c
void UpdateWeight (const Vec& grad, float eta, float lambda, Vec& weight) {
      weight = -eta * (grad + lambda * weight);
} 
```
答案是肯定的，但不是最明显的解决方案。

## 一个Naive的解决方案
让我们先来看看一个最直接的解决方案：操作符重载。
```c
// Naive solution for vector operation overloading 
struct Vec {
  int len;
  float* dptr;
  Vec(int len) : len(len) { 
    dptr = new float[len];
  }
  Vec(const Vec& src) : len(src.len) {
    dptr = new float[len];
    memcpy(dptr, src.dptr, sizeof(float)*len ); 
  }
  ~Vec(void) {
    delete [] dptr;
  }
};

inline Vec operator+(const Vec &lhs, const Vec &rhs) {
  Vec res(lhs.len);
  for (int i = 0; i < lhs.len; ++i) {
    res.dptr[i] = lhs.dptr[i] + rhs.dptr[i];
  } 
  return res;
} 
```

如果我们以相同的风格添加更多的运算符重载，我们可以得到我们想要的，并写成表达式，而不是循环。然而，这种方法是低效的，因为临时内存在每个操作期间被分配和释放，而我们可以做得更好。  

一个替代的，更有效的方法是只重载operator + =，operator- =，这可以在没有临时内存分配的情况下实现。但这限制了我们可以写的方程。  

我们将讨论为什么我们仍然需要表达式模板，尽管C ++ 11在本教程结尾处提供了移动赋值运算符和右值引用。  

## 惰性求值
让我们想想为什么我们在做operator +时需要临时内存分配。这是因为我们不知道将在operator +中赋值的目标，否则我们可以直接存储到目标内存而不是临时内存中。  

如果我们能知道目标怎么办？下面的代码（exp_lazy.cpp）实现了这一点。
```c
// Example Lazy evaluation code
// for simplicity, we use struct and make all members public
#include <cstdio>
struct Vec;
// expression structure holds the expression
struct BinaryAddExp {
  const Vec &lhs;
  const Vec &rhs;
  BinaryAddExp(const Vec &lhs, const Vec &rhs)
  : lhs(lhs), rhs(rhs) {}
};
// no constructor and destructor to allocate and de-allocate memory,
//  allocation done by user
struct Vec {
  int len;
  float* dptr;
  Vec(void) {}
  Vec(float *dptr, int len)
      : len(len), dptr(dptr) {}
  // here is where evaluation happens
  inline Vec &operator=(const BinaryAddExp &src) {
    for (int i = 0; i < len; ++i) {
      dptr[i] = src.lhs.dptr[i] + src.rhs.dptr[i];
    }
    return *this;
  }
};
// no evaluation happens here
inline BinaryAddExp operator+(const Vec &lhs, const Vec &rhs) {
  return BinaryAddExp(lhs, rhs);
}

const int n = 3;
int main(void) {
  float sa[n] = {1, 2, 3};
  float sb[n] = {2, 3, 4};
  float sc[n] = {3, 4, 5};
  Vec A(sa, n), B(sb, n), C(sc, n);
  // run expression
  A = B + C;
  for (int i = 0; i < n; ++i) {
    printf("%d:%f==%f+%f\n", i, A.dptr[i], B.dptr[i], C.dptr[i]);
  }
  return 0;
}
```
这个想法是，我们实际上不是在operator +中执行计算，而是只返回一个表达式结构（如抽象语法树），当我们重载operator =时，我们看到目标，以及所有的操作数，我们可以运行计算无需引入额外的内存！类似地，我们可以定义一个DotExp并且在operator =处进行延迟计算，并将矩阵（向量）乘法重定向到BLAS。

## 更长的表达式和表达式模板
通过使用惰性求值，我们通过避免临时内存分配很酷。但是代码的能力有限：

* 我们只能写A = B + C，但不能更长的表达式。
* 当我们添加更多的表达式时，我们需要写更多的operator =来计算每个方程。

这里需要模板编程的魔力来救援。下面的代码（exp_template.cpp），有点更长，也允许你写长的方程。
```c
// Example code, expression template, and more length equations
// for simplicity, we use struct and make all members public
#include <cstdio>

// this is expression, all expressions must inheritate it,
//  and put their type in subtype
template<typename SubType>
struct Exp {
  // returns const reference of the actual type of this expression
  inline const SubType& self(void) const {
    return *static_cast<const SubType*>(this);
  }
};

// binary add expression
// note how it is inheritates from Exp
// and put its own type into the template argument
template<typename TLhs, typename TRhs>
struct BinaryAddExp: public Exp<BinaryAddExp<TLhs, TRhs> > {
  const TLhs &lhs;
  const TRhs &rhs;
  BinaryAddExp(const TLhs& lhs, const TRhs& rhs)
      : lhs(lhs), rhs(rhs) {}
  // evaluation function, evaluate this expression at position i
  inline float Eval(int i) const {
    return lhs.Eval(i) + rhs.Eval(i);
  }
};
// no constructor and destructor to allocate
// and de-allocate memory, allocation done by user
struct Vec: public Exp<Vec> {
  int len;
  float* dptr;
  Vec(void) {}
  Vec(float *dptr, int len)
      :len(len), dptr(dptr) {}
  // here is where evaluation happens
  template<typename EType>
  inline Vec& operator= (const Exp<EType>& src_) {
    const EType &src = src_.self();
    for (int i = 0; i < len; ++i) {
      dptr[i] = src.Eval(i);
    }
    return *this;
  }
  // evaluation function, evaluate this expression at position i
  inline float Eval(int i) const {
    return dptr[i];
  }
};
// template add, works for any expressions
template<typename TLhs, typename TRhs>
inline BinaryAddExp<TLhs, TRhs> operator+(const Exp<TLhs> &lhs, const Exp<TRhs> &rhs) {
  return BinaryAddExp<TLhs, TRhs>(lhs.self(), rhs.self());
}

const int n = 3;
int main(void) {
  float sa[n] = {1, 2, 3};
  float sb[n] = {2, 3, 4};
  float sc[n] = {3, 4, 5};
  Vec A(sa, n), B(sb, n), C(sc, n);
  // run expression, this expression is longer:)
  A = B + C + C;
  for (int i = 0; i < n; ++i) {
    printf("%d:%f == %f + %f + %f\n", i,
           A.dptr[i], B.dptr[i],
           C.dptr[i], C.dptr[i]);
  }
  return 0;
}
```
代码的关键思想是模板Exp <SubType>将其派生类的类型作为模板参数，因此它可以通过self（）将自身转换为SubType。 BinaryAddExp现在是一个模板类，可以将表达式复合在一起，像模板版本的复合模式。求值是通过函数Eval完成的，这是在BinaryAddExp中以递归方式完成的。

* 由于内联，在编译时将在operator =中的src.Eval（i）的函数调用编译成B.dptr [i] + C.dptr [i] + C.dptr [i]。
* 我们可以用相同的效率编写element-wise操作的方程，就像我们写一个循环

## 使其更灵活
正如我们在前面的例子中可以看到的，模板编程是一个强大的功能，使编译时更灵活，我们的最后一个例子，更接近mshadow，允许用户自定义二元运算符（exp_template_op.cpp）。
```c
// Example code, expression template
// with binary operator definition and extension
// for simplicity, we use struct and make all members public
#include <cstdio>

// this is expression, all expressions must inheritate it,
// and put their type in subtype
template<typename SubType>
struct Exp{
  // returns const reference of the actual type of this expression
  inline const SubType& self(void) const {
    return *static_cast<const SubType*>(this);
  }
};

// binary operators
struct mul{
  inline static float Map(float a, float b) {
    return a * b;
  }
};

// binary add expression
// note how it is inheritates from Exp
// and put its own type into the template argument
template<typename OP, typename TLhs, typename TRhs>
struct BinaryMapExp: public Exp<BinaryMapExp<OP, TLhs, TRhs> >{
  const TLhs& lhs;
  const TRhs& rhs;
  BinaryMapExp(const TLhs& lhs, const TRhs& rhs)
      :lhs(lhs), rhs(rhs) {}
  // evaluation function, evaluate this expression at position i
  inline float Eval(int i) const {
    return OP::Map(lhs.Eval(i), rhs.Eval(i));
  }
};
// no constructor and destructor to allocate and de-allocate memory
// allocation done by user
struct Vec: public Exp<Vec>{
  int len;
  float* dptr;
  Vec(void) {}
  Vec(float *dptr, int len)
      : len(len), dptr(dptr) {}
  // here is where evaluation happens
  template<typename EType>
  inline Vec& operator=(const Exp<EType>& src_) {
    const EType &src = src_.self();
    for (int i = 0; i < len; ++i) {
      dptr[i] = src.Eval(i);
    }
    return *this;
  }
  // evaluation function, evaluate this expression at position i
  inline float Eval(int i) const {
    return dptr[i];
  }
};
// template add, works for any expressions
template<typename OP, typename TLhs, typename TRhs>
inline BinaryMapExp<OP, TLhs, TRhs>
F(const Exp<TLhs>& lhs, const Exp<TRhs>& rhs) {
  return BinaryMapExp<OP, TLhs, TRhs>(lhs.self(), rhs.self());
}

template<typename TLhs, typename TRhs>
inline BinaryMapExp<mul, TLhs, TRhs>
operator*(const Exp<TLhs>& lhs, const Exp<TRhs>& rhs) {
  return F<mul>(lhs, rhs);
}

// user defined operation
struct maximum{
  inline static float Map(float a, float b) {
    return a > b ? a : b;
  }
};

const int n = 3;
int main(void) {
  float sa[n] = {1, 2, 3};
  float sb[n] = {2, 3, 4};
  float sc[n] = {3, 4, 5};
  Vec A(sa, n), B(sb, n), C(sc, n);
  // run expression, this expression is longer:)
  A = B * F<maximum>(C, B);
  for (int i = 0; i < n; ++i) {
    printf("%d:%f == %f * max(%f, %f)\n",
           i, A.dptr[i], B.dptr[i], C.dptr[i], B.dptr[i]);
  }
  return 0;
}
```


## 总结
到目前为止，你应该已经了解它的工作原理：

* 惰性求值，让我们看到所有的操作数和目标
* 模板组合和递归求值，允许我们评估任意使用element-wise操作的复合表达式。
* 由于模板和内联，写表达式就像我们直接编写for循环来实现更新规则一样高效:)

所以在写机器学习代码时写表达式，并将重点放在重要的算法部分。

## MShadow中的表达式模板
mshadow中的表达式模板使用与教程中介绍的相同的要点，但有一些细微的差别：

* 我们将评估代码与表达式构造和组合代码分开。
    * 而不是将Eval放在Exp类中。从表达式创建计划类，并用于评估结果。
    * 这允许我们在Plan中放置较少的变量，例如，当我们评估数据时，我们不需要数组长度。
    * 一个重要的原因是CUDA内核不能带有const引用的类
    * 这个设计选择是有争议的，但我们发现它到目前为止是有用的。
* 对矩阵点乘等复杂表达式的惰性支持
    * 除了element-wise的表达式，我们还想支持语法糖，如A = dot（B.T（），C），再次，惰性求值没有额外的内存分配。
* 类型检查和数组长度检查。

## 备注

* 表达式模板和C ++ 11：在C ++ 11中，move构造函数可以用来保存重复分配内存，这消除了对表达式模板的一些需要。然而，该空间仍然需要至少分配一次。
* 这只消除了表达式模板然后表达式生成空间的需要，说dst = A + B + C，dst不包含赋值前分配的空间。
* 如果我们想要保留一切都预先分配的语法，并且表达式执行没有内存分配（这是我们在mshadow中做的），我们仍然需要表达式模板。

原文：  
https://github.com/dmlc/mshadow/tree/master/guide/exp-template